<a href="https://colab.research.google.com/github/kavyasreemaniga/DATA_690/blob/main/Big_Data_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 439 kB 4.8 MB/s 


## Write three Python scripts using mrjob that tell me:
1. Average number of words in each review (define “words”
however you like but be explicit about it) ( 5 marks )

In [14]:
from IPython.utils import text
import csv

filename = open('yelp.csv','r')

file = csv.DictReader(filename)

review = []

for col in file:
  review.append(col['text'])

print('text:',text)


text: <module 'IPython.utils.text' from '/usr/local/lib/python3.7/dist-packages/IPython/utils/text.py'>


In [15]:
print(len(review))

10000


In [16]:
with open(r'reviewfile.txt','w') as fp:
  for item in review:
    # write each item on a new line
    fp.write("%s\n" % item)
  print('Done')

Done


In [17]:
%%file wc.py
##First install mrjob by running:
from mrjob.job import MRJob

##First we create a class called MRWordFrequencyCount, remember to put ‘MRJob’ in the argument. 
#Then we create a mapper function and a reducer function inside the class

class MRWordFrequencyCount(MRJob):
##Note that (_, line) tells MRJob to ignore the key and take each line of the document as the value.
    def mapper(self, _, line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1
#Inside the reducer function, our (key, value) input pairs are (word, count) and our output pairs should be (word, sum(count)).
    def reducer(self, key, values):
        yield key, sum(values)
### Warning: do not forget these two lines,##or else your script will not run!

if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting wc.py


In [19]:
import wc

mr_job = wc.MRWordFrequencyCount(args=['reviewfile.txt'])
with mr_job.make_runner() as runner:
     runner.run()
     for key, value in mr_job.parse_output(runner.cat_output()):
       print(key, value)

words 1310396
chars 7107387
lines 10000


## 2. Count of reviews by year-month (eg “2021-09”) ( 5 marks )

In [6]:
# Creating list with date column
from IPython.utils import text
import csv

filename = open('yelp.csv','r')

file = csv.DictReader(filename)

date = []

for col in file:
  date.append(col['date'])

print('date:',date)

date: ['2011-01-26', '2011-07-27', '2012-06-14', '2010-05-27', '2012-01-05', '2007-12-13', '2010-02-12', '2012-07-12', '2012-08-17', '2010-08-11', '2010-06-16', '2011-10-21', '2010-01-11', '2011-12-23', '2010-05-20', '2011-03-20', '2008-10-12', '2010-05-03', '2009-03-06', '2011-11-17', '2008-10-08', '2011-06-28', '2011-07-13', '2010-09-05', '2011-05-22', '2010-05-26', '2013-01-03', '2008-03-21', '2011-03-30', '2012-07-12', '2011-05-09', '2012-05-04', '2012-09-09', '2010-03-28', '2011-03-27', '2011-11-30', '2008-12-28', '2012-10-16', '2011-01-02', '2008-07-09', '2011-11-09', '2009-03-13', '2012-06-30', '2012-04-12', '2009-06-13', '2011-12-21', '2009-04-04', '2010-09-25', '2012-01-03', '2011-10-03', '2010-08-04', '2010-02-01', '2007-10-29', '2011-12-13', '2009-08-06', '2012-01-14', '2009-09-02', '2010-02-04', '2010-04-24', '2010-09-28', '2012-06-08', '2011-11-22', '2008-01-28', '2010-02-05', '2012-05-19', '2011-05-04', '2011-12-30', '2011-04-28', '2011-08-14', '2012-01-05', '2010-05-22',

In [20]:
import pandas as pd

lst = pd.DataFrame({'Date': date,'Review': review})
lst

,Date,Review
0,2011-01-26,My wife took me here on my birthday for breakf...
1,2011-07-27,I have no idea why some people give bad review...
2,2012-06-14,love the gyro plate. Rice is so good and I als...
3,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,2012-01-05,General Manager Scott Petello is a good egg!!!...
...,...,...
9995,2012-07-28,First visit...Had lunch here today - used my G...
9996,2012-01-18,Should be called house of deliciousness! I co...
9997,2010-11-16,I recently visited Olive and Ivy for business ...
9998,2012-12-02,My nephew just moved to Scottsdale recently so...


In [21]:
lst.sort_values(by='Date', inplace = True)

print(lst)

            Date                                             Review
7945  2005-04-18  Wood fired pizzas, scrumptious pasta dishes an...
1817  2005-07-03  Happy hour in Phoenix from 5PM to midnight on ...
9743  2005-07-10  You're in 120 degree Scottsdale weather and yo...
4383  2005-12-16  decent.  the potstickers were kind of good.  I...
7170  2006-01-08  I went here because all my friends were peeing...
...          ...                                                ...
6113  2013-01-05  Great food. Service consistently blows. Sit at...
2620  2013-01-05  I was a little bit scared to eat at this place...
633   2013-01-05  My husband surprised me with a stay-cation at ...
2264  2013-01-05  It appears that the service kinks have been wo...
2355  2013-01-05  My daughter and I really love it here.  I alwa...

[10000 rows x 2 columns]


In [24]:
print(type(lst.Date[0]))

lst['Date'] = pd.to_datetime(lst['Date'])

print(type(lst.Date[0]))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [34]:
print(lst.groupby('Date').size())

Date
2005-04-18     1
2005-07-03     1
2005-07-10     1
2005-12-16     1
2006-01-08     1
              ..
2013-01-01     9
2013-01-02    15
2013-01-03    10
2013-01-04    12
2013-01-05     6
Length: 1995, dtype: int64


#### count of reviews per year

In [26]:
lst['Date'].groupby(lst.Date.dt.year).agg('count')

Date
2005       4
2006      55
2007     285
2008     765
2009    1171
2010    1852
2011    2791
2012    3025
2013      52
Name: Date, dtype: int64

In [28]:
lst['Date'].groupby([lst.Date.dt.year, lst.Date.dt.month]).agg('count')

Date  Date
2005  4         1
      7         2
      12        1
2006  1         6
      2         9
             ... 
2012  9       239
      10      258
      11      208
      12      196
2013  1        52
Name: Date, Length: 87, dtype: int64

In [35]:
lst.groupby([lst['Date'].dt.year.rename('year'), lst['Date'].dt.month.rename('month')]).agg({'count'})

Date Review
           count  count
year month             
2005 4         1      1
     7         2      2
     12        1      1
2006 1         6      6
     2         9      9
...          ...    ...
2012 9       239    239
     10      258    258
     11      208    208
     12      196    196
2013 1        52     52

[87 rows x 2 columns]

### 3. Average rating of any review marked ”cool” (eg where cool!= 0)

In [37]:
# Creating list with date column
from IPython.utils import text
import csv

filename = open('yelp.csv','r')

file = csv.DictReader(filename)

rating = []

for col in file:
  rating.append(col['cool'])

print('rating:',rating)

rating: ['2', '0', '0', '1', '0', '4', '7', '0', '0', '0', '1', '1', '1', '1', '1', '0', '3', '0', '5', '1', '2', '1', '0', '0', '0', '0', '0', '2', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '2', '0', '0', '1', '0', '2', '0', '0', '1', '1', '3', '1', '0', '2', '0', '1', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '3', '0', '1', '0', '1', '0', '0', '0', '11', '0', '1', '0', '2', '0', '0', '0', '7', '0', '2', '3', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '6', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '8', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '16', '0', '2', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '2', '1', '0', '4', '4', '0', '1', '0', '0', '2', '0', '0', '0', '0', '16', '0', '0', '2', '0', '2', '28', '1', '0', '0', '0', '0', '0', '1', '0', '0', '4', '0', '0', '1', '1', '0', '0', '1

In [38]:
my_file = pd.read_csv("yelp.csv")

my_file['cool'].mean()

0.8768